In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv("./exported_data.csv",sep=',', decimal=",") # Считываем данные из файла
data.head()  


,Bought_Or_Not,Recency,Frequency,Monetary_Value,Mean_Quantity,ActivationDate,Age,SubscribedEmail,SubscribedPush,Male
0,0.0,160.0,8.142857142857142,251.78947368421052,1.0350877192982457,735599,24.0,False,True,0
1,0.0,720.0,4.857142857142857,279.6470588235294,1.0,735599,46.0,True,True,1
2,1.0,15.0,4.714285714285714,398.7348484848485,1.6666666666666667,735599,27.0,True,True,0
3,0.0,177.0,14.142857142857142,262.7373737373738,1.1414141414141414,735599,54.0,True,True,0
4,1.0,115.0,3.0,393.7142857142857,1.8095238095238095,735599,48.0,False,False,1


In [ ]:

from sklearn.tree import DecisionTreeClassifier
# ДЕЛИМ ДАННЫЕ
X=data.drop('Bought_Or_Not',axis=1) # исключаем из данных зависимую переменную
y=data['Bought_Or_Not']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
tree = DecisionTreeClassifier(max_depth=6,criterion='entropy', random_state=1)

# ОБУЧАЕМ ДЕРЕВО
tree.fit(X_train, y_train)

# ПРЕДСКАЗЫВАЕМ
y_pred = tree.predict(X_test)
print(y_pred[0:5])

# СЧИТАЕМ ВСЯКИЕ МЕТРИКИ
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE

acc = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(acc))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
#  mse_TREE
mse_tree = MSE(y_test, y_pred)

#  rmse_TREE
rmse_tree = mse_tree**(1/2)

# Print rmse_dt
print("Test set RMSE of tree: {:.2f}".format(rmse_tree))
# СТРОИМ ГРАФИК
plot_labeled_decision_regions(X_test, y_test, tree)